**Saad Khan and Nithun Selva**

Spring 2025

CS 444: Deep Learning

Project 1: Deep Neural Networks 

#### Week 1: VGG4 and building a deep learning library

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.rcParams.update({'font.size': 18})

np.set_printoptions(suppress=True, precision=4)

# Automatically reload your external source code
%load_ext autoreload
%autoreload 2

2025-02-09 17:16:26.013422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-09 17:16:26.024883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739139386.040487    7105 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739139386.045731    7105 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 17:16:26.061770: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Task 4. Train `VGG4` on MNIST and CIFAR-10

Woohoo! VGG4 is built and tested! Time for some fun! 😎

To train in a reasonable amount of time, upload your project code to CoCalc so that you can train on the GPU. See [instructions on the class website ](https://cs.colby.edu/courses/S25/cs444/software.html#cocalc) for getting this setup.

*As noted on the website, if you have a higher-end Macbook with a Pro or Max chip and ≥ 16 GB of memory, you could probably run this workload on your computer if you would prefer. This is entirely optional and the cloud will almost certainly be faster than even the fastest Mac. Setting this up would just provide some extra convenience/flexibility. See the [macOS instructions](https://cs.colby.edu/courses/S25/cs444/software.html#tfmac) for setup.*

Run the cell below to make sure TensorFlow is running on the GPU.

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if len(gpus) > 0:
    print('Running on the GPU')
else:
    print('NOT running on the GPU')

Running on the GPU


In [3]:
from datasets import get_dataset
from vgg_nets import VGG4

### 4a. Train `VGG4` on MNIST on the GPU

This will be a "hello world" test to make sure your `fit` method is working.

Write code in the cell below to load in MNIST

In [4]:
#load mnist data
x_train, y_train, x_val, y_val, x_test, y_test, classnames = get_dataset("mnist")




# KEEP ME
print(f'Your training set data have shape {x_train.shape} and they should be (54000, 28, 28, 1)')
print(f'Your training set labels have shape {y_train.shape} and they should be (54000,)')
print(f'Your val set data have shape {x_val.shape} and they should be (6000, 28, 28, 1)')
print(f'Your val set labels have shape {y_val.shape} and they should be (6000,)')
print(f'Your test set data have shape {x_test.shape} and they should be (10000, 28, 28, 1)')
print(f'Your test set labels have shape {y_test.shape} and they should be (10000,)')

I0000 00:00:1739139387.882784    7105 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6


Your training set data have shape (54000, 28, 28, 1) and they should be (54000, 28, 28, 1)
Your training set labels have shape (54000,) and they should be (54000,)
Your val set data have shape (6000, 28, 28, 1) and they should be (6000, 28, 28, 1)
Your val set labels have shape (6000,) and they should be (6000,)
Your test set data have shape (10000, 28, 28, 1) and they should be (10000, 28, 28, 1)
Your test set labels have shape (10000,) and they should be (10000,)


Train `VGG4` in the cell below on MNIST for `7` epochs and a batch size of `1024`! Print out your accuracy on the test set when training is done.

The **entire** process of training and evaluating test accuracy should take no more than 1 min (*at most!*). If it is taking longer, seek help.

Your print outs should look something like:

```
---------------------------------------------------------------------------
Dense layer output(output) shape: [1, 10]
Dropout layer output(dropout1) shape: [1, 128]
Dense layer output(dense1) shape: [1, 128]
Flatten layer output(flat) shape: [1, 12544]
MaxPool2D layer output(maxpool1) shape: [1, 14, 14, 64]
Conv2D layer output(conv2) shape: [1, 28, 28, 64]
Conv2D layer output(conv1) shape: [1, 28, 28, 64]
---------------------------------------------------------------------------
Epoch 0/6, Training loss 1.54, Val loss 0.33, Val acc 89.94 
Epoch 0 took: blah0 secs
Epoch 1/6, Training loss 0.42, Val loss 0.17, Val acc 94.59 
Epoch 1 took: blah1 secs
...
Epoch 6/6, Training loss surprise, Val loss surprise, Val acc surprise
Epoch 6 took: blah6 secs
Finished training after 7 epochs!
VGG4 MNIST Test accuracy: surprise%
```

The val and test accuracy should be satisfyingly high — in the high 90s.

In [ ]:
# KEEP THIS SEED
tf.random.set_seed(0)

#train vgg4 model on mnist for 7 epochs with batch size 1024, report the accuracy on the test set
model = VGG4(C=10, input_feats_shape=(28, 28, 1))
model.compile(optimizer='adam', loss='cross_entropy')
train_loss_hist, val_loss_hist, val_acc_hist, e = model.fit(x_train, y_train, x_val, y_val, max_epochs=7, batch_size=1024)

#calculate the accuracy on the test set
model.set_layer_training_mode(False)  # Disable dropout & batch norm during evaluation
test_acc, test_loss = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 10]
Dropout layer output(Dropout) shape: [1, 128]
Dense layer output(Dense_1) shape: [1, 128]
Flatten layer output(Flatten) shape: [1, 12544]
MaxPool2D layer output(MaxPool_1) shape: [1, 14, 14, 64]
Conv2D layer output(Conv2D_2) shape: [1, 28, 28, 64]
Conv2D layer output(Conv2D_1) shape: [1, 28, 28, 64]
---------------------------------------------------------------------------
Epoch 1/7 - Train Loss: 1.5436, Val Loss: 0.3262, Val Acc: 0.8985
Epoch 1 completed in 3.28 seconds.
Epoch 2/7 - Train Loss: 0.2725, Val Loss: 0.1527, Val Acc: 0.9546
Epoch 2 completed in 2.03 seconds.
Epoch 3/7 - Train Loss: 0.1668, Val Loss: 0.1302, Val Acc: 0.9600
Epoch 3 completed in 2.00 seconds.
Epoch 4/7 - Train Loss: 0.1277, Val Loss: 0.1058, Val Acc: 0.9694
Epoch 4 completed in 2.02 seconds.
Epoch 5/7 - Train Loss: 0.1063, Val Loss: 0.0971, Val Acc: 0.9709
Epoch 5 completed in 1.98 seconds.


### 4b. Train `VGG4` on MNIST on the CPU

To appreciate the advantage for training deep networks on GPUs instead of your computer's CPU, copy-paste your code above that trains your VGG4 net on MNIST below. Instead of running it on CoCalc, run it locally on your computer. Be sure to print out the time per epoch.

*If the net takes >2 minutes per epoch on your computer, just train for one epoch then call it quits :)*

### 4c. Questions

**Question 1:** Approximately how long did it take for 1 epoch of training with and without the GPU. Compute the relative compute time (`gpu_time_per_epoch`/`cpu_time_per_epoch`). What do you think?

**Answer 1:**

### 4d. Train `VGG4` on CIFAR-10

Now let's train on CIFAR-10. Run this (*and all subsequent large training sessions*) on CoCalc/the GPU 😊

Write code in the cell below to load in CIFAR-10.

In [7]:

# KEEP ME
print(f'Your training set data have shape {x_train.shape} and they should be (45000, 32, 32, 3)')
print(f'Your training set labels have shape {y_train.shape} and they should be (45000,)')
print(f'Your val set data have shape {x_val.shape} and they should be (5000, 32, 32, 3)')
print(f'Your val set labels have shape {y_val.shape} and they should be (5000,)')
print(f'Your test set data have shape {x_test.shape} and they should be (10000, 32, 32, 3)')
print(f'Your test set labels have shape {y_test.shape} and they should be (10000,)')

Your training set data have shape (54000, 28, 28, 1) and they should be (45000, 32, 32, 3)
Your training set labels have shape (54000,) and they should be (45000,)
Your val set data have shape (6000, 28, 28, 1) and they should be (5000, 32, 32, 3)
Your val set labels have shape (6000,) and they should be (5000,)
Your test set data have shape (10000, 28, 28, 1) and they should be (10000, 32, 32, 3)
Your test set labels have shape (10000,) and they should be (10000,)


Train `VGG4` on CIFAR-10 in the cell below for `15` epochs and use a batch size of `128`.

*You should anticipate 10-20 secs per epoch of training. Thus, the whole training session should take ~2.5-5 mins. If this is far off, please seek help.*

Plot the training and val loss over epochs. Put test acc in the title. *If everything is working, your training and validation loss should steadily decrease then start to plateau by the end of training.*

In [8]:
# KEEP THIS SEED
tf.random.set_seed(0)



### 4e. Experiment: Effect of batch size on runtime and accuracy

To develop your intuition about how the choice of batch size generally affects runtime and accuracy on a dataset like CIFAR-10, run an experiment in which fresh `VGG4` nets are trained for `15` epochs with one of the following batch sizes:<br/>
`[128, 256, 512, 1024, 2048]`. After each training run, record the test accuracy and runtime.

Create two plots:
1. The test accuracy (y axis) as a function of the batch size (x axis). There should be 5 markers joined by a single curve.
2. The runtime (y axis) as a function of the batch size (x axis). There should be 5 markers joined by a single curve.

**Note:**
- Your `fit` method prints the runtime per epoch, but in this task you should record the total runtime over training and prediction. (in seconds). To do this, it may make sense to use the time module to record the total time in the notebook cell below.
- You should be running this on the GPU!

In [9]:
import time

In [10]:
# KEEP THIS SEED
tf.random.set_seed(0)


### 4f. Questions

**Question 2:** What do the plots suggest to you about the relationship between batch size and accuracy? Please be specific,citing evidence from your plots.

**Question 3:** Do you find this relationship surprising? Why or why not?

**Answer 2:** 

**Answer 3:** 